In [1]:
from utils import extract_image_info, split_channels, segment_nuclei, extract_intensities
from tqdm import tqdm
import pandas as pd
import os
from pathlib import Path
from cellpose import models
import plotly.express as px

In [2]:
# -------- DEFINE YOUR ANALYSIS PARAMETERS BELOW --------- #

# This is the default Cellpose model to segment nuclei, it leverages a GPU if its available
model = models.Cellpose(gpu=True, model_type="nuclei")

# Diameter in pixels of the nuclei present in your image, helps Cellpose to adjust nuclei mask predictions
cellpose_nuclei_diameter = 40

# Blurs the mip_nuclei image to even out high intensity foci within the nucleus, the higher the value blurriness increases
# High values help segment sparse nuclei (CA and CTX regions) but as a drawback it merges nuclei entities that are very close together (DG region)
gaussian_sigma = 5

# Scan for all images present in the data folder and add their Paths to the images list
data_path = Path("./data")
images = []

for file_path in data_path.glob("**/**/**/*.lsm"):
    images.append(file_path)

In [3]:
# ------- ANALYSIS PIPELINE --------- #

# Create an empty list to append the resulting dataframes from each analysis round
dataframes = []

for image in tqdm(images):

    # Extract filename, region, mouse and IHC round
    filename, region, mouse_id, ihc_round = extract_image_info(image)

    # Split channels
    nuclei_img, h2a_img, cfos_img = split_channels(image)

    # Segment nuclei
    nuclei_masks = segment_nuclei(nuclei_img, gaussian_sigma, model, cellpose_nuclei_diameter)

    # Morphological and intensity measurements
    merged_df = extract_intensities(nuclei_masks, h2a_img, cfos_img, filename, region, mouse_id, ihc_round)
    
    dataframes.append(merged_df)
    
# Concatenate all DataFrames in the list into a single DataFrame
final_df = pd.concat(dataframes, ignore_index=True)

# Define output folder for results
results_folder = "./results/"

# Create the necessary folder structure if it does not exist
try:
    os.mkdir(str(results_folder))
    print(f"Output folder created: {results_folder}")
except FileExistsError:
    print(f"Output folder already exists: {results_folder}")
    
# Saves a copy of final_df as a .csv file
final_df.to_csv(f"./results/results_cellpdia{cellpose_nuclei_diameter}_sigma{gaussian_sigma}.csv")


100%|██████████| 264/264 [16:41<00:00,  3.79s/it]


Output folder created: ./results/


In [4]:
final_df.head()

,filename,region,mouse_id,ihc_round,label,cfos_intensity_mean,cfos_intensity_max,h2a_intensity_mean,h2a_intensity_max,area_filled,perimeter,equivalent_diameter
0,Image1,CA1,AD1867,IHC_1,1,13.266192,44.0,13.809585,58.0,1544.0,155.982756,44.338266
1,Image1,CA1,AD1867,IHC_1,2,11.156463,21.0,3.823129,22.0,147.0,62.727922,13.680870
2,Image1,CA1,AD1867,IHC_1,3,18.486083,38.0,22.911745,83.0,1473.0,152.669048,43.306834
3,Image1,CA1,AD1867,IHC_1,4,18.557368,60.0,10.042958,62.0,1839.0,166.124892,48.388919
4,Image1,CA1,AD1867,IHC_1,5,18.387984,35.0,12.437882,59.0,982.0,120.568542,35.359882


In [5]:
# Create a histogram of cfos_intensity_mean values
fig = px.histogram(final_df, x='h2a_intensity_mean', nbins=50, title='Distribution of H2A Intensity Mean')

# Update layout if necessary
fig.update_layout(
    xaxis_title='H2A Intensity Mean',
    yaxis_title='Count',
    bargap=0.2
)

# Show the plot
fig.show()

In [6]:
# Create a histogram of cfos_intensity_mean values
fig = px.histogram(final_df, x='cfos_intensity_mean', nbins=50, title='Distribution of CFOS Intensity Mean')

# Update layout if necessary
fig.update_layout(
    xaxis_title='CFOS Intensity Mean',
    yaxis_title='Count',
    bargap=0.2
)

# Show the plot
fig.show()